## Training process for ECD507 Senior Capstone Project - F1Tenth ML Based Autonomous Race Car
### Contributors - Charles Hodgins, Rishabh Hegde, Dylan DiGiacomo, and Andrew Meccariello

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
print(torch.cuda.is_available())
print(torch.version.cuda)


True
12.6


In [7]:
class DrivingDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx,1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (200, 66))  # Resize using OpenCV
        image = image / 255.0  # Normalize pixel values to [0,1]
        image = np.transpose(image, (2, 0, 1))  # Change shape to (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        steering_angle = float(self.data.iloc[idx, 3])
        throttle = float(self.data.iloc[idx, 4])
               
        return image, torch.tensor([steering_angle, throttle], dtype=torch.float32)


In [ ]:
dataset = DrivingDataset('data/driving_log.csv')
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

(tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000],
          [0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000],
          [0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000],
          ...,
          [0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000],
          [0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000],
          [0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ...,

In [6]:
# Define CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3),
            nn.ReLU()
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3840, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 10),
            nn.ReLU(),
            nn.Linear(10, 2)  # Output: Steering angle and Throttle
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [ ]:
# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # change to cuda when it begins to work
print(device)
model = CNNModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for images, targets in dataloader:
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}')

# Save model
torch.save(model.state_dict(), os.path.expanduser('~/f1tenth_ws/model.pth'))
print("Model training complete and saved as model.pth")

cuda
Epoch 1/10, Loss: 0.004548632423393428
Epoch 2/10, Loss: 0.0012387587339617312
Epoch 3/10, Loss: 0.0004631531337508932
Epoch 4/10, Loss: 0.00034348509507253766
Epoch 5/10, Loss: 8.241189971158747e-05
Epoch 6/10, Loss: 0.00012421612336765975
Epoch 7/10, Loss: 6.578569536941359e-05
Epoch 8/10, Loss: 8.121327709886828e-06
Epoch 9/10, Loss: 4.321260712458752e-05
Epoch 10/10, Loss: 3.734299934876617e-05
Model training complete and saved as model.pth


In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.backends.cudnn.enabled)  # Should return True
print(torch.version.cuda)

print(model.state_dict())


True
Orin
True
12.6


In [9]:
img, label = dataset[20]
img = img.to(device)
img = img.unsqueeze(0)  # Add batch dim
with torch.no_grad():
    output = model(img)

print(output)

tensor([[-0.0048, -0.0024]], device='cuda:0')
